# EDA
Files:
- game.csv
- game_teams_stats.csv
- game_skater_stats.csv
- game_goalie_stats.csv
- player_info.csv

In [1]:
import pandas as pd
import numpy as np
import copy
import math

In [2]:
data_path = 'nhl_data/'

In [3]:
games = pd.read_csv(data_path+'game.csv')
print(games.shape)
games.head()

(26305, 15)


,game_id,season,type,date_time_GMT,away_team_id,home_team_id,away_goals,home_goals,outcome,home_rink_side_start,venue,venue_link,venue_time_zone_id,venue_time_zone_offset,venue_time_zone_tz
0,2016020045,20162017,R,2016-10-19T00:30:00Z,4,16,4,7,home win REG,right,United Center,/api/v1/venues/null,America/Chicago,-5,CDT
1,2017020812,20172018,R,2018-02-07T00:00:00Z,24,7,4,3,away win OT,left,KeyBank Center,/api/v1/venues/null,America/New_York,-4,EDT
2,2015020314,20152016,R,2015-11-24T01:00:00Z,21,52,4,1,away win REG,right,MTS Centre,/api/v1/venues/null,America/Winnipeg,-5,CDT
3,2015020849,20152016,R,2016-02-17T00:00:00Z,52,12,1,2,home win REG,right,PNC Arena,/api/v1/venues/null,America/New_York,-4,EDT
4,2017020586,20172018,R,2017-12-30T03:00:00Z,20,24,1,2,home win REG,left,Honda Center,/api/v1/venues/null,America/Los_Angeles,-7,PDT


In [4]:
game_stats = pd.read_csv(data_path+'game_teams_stats.csv')
print(game_stats.shape)
game_stats.head()

(52610, 17)


,game_id,team_id,HoA,won,settled_in,head_coach,goals,shots,hits,pim,powerPlayOpportunities,powerPlayGoals,faceOffWinPercentage,giveaways,takeaways,blocked,startRinkSide
0,2016020045,4,away,False,REG,Dave Hakstol,4.0,27.0,30.0,6.0,4.0,2.0,50.9,12.0,9.0,11.0,left
1,2016020045,16,home,True,REG,Joel Quenneville,7.0,28.0,20.0,8.0,3.0,2.0,49.1,16.0,8.0,9.0,left
2,2017020812,24,away,True,OT,Randy Carlyle,4.0,34.0,16.0,6.0,3.0,1.0,43.8,7.0,4.0,14.0,right
3,2017020812,7,home,False,OT,Phil Housley,3.0,33.0,17.0,8.0,2.0,1.0,56.2,5.0,6.0,14.0,right
4,2015020314,21,away,True,REG,Patrick Roy,4.0,29.0,17.0,9.0,3.0,1.0,45.7,13.0,5.0,20.0,left


In [5]:
game_skater_stats = pd.read_csv(data_path+'game_skater_stats.csv')
print(game_skater_stats.shape)
game_skater_stats.head()

(945830, 22)


,game_id,player_id,team_id,timeOnIce,assists,goals,shots,hits,powerPlayGoals,powerPlayAssists,...,faceoffTaken,takeaways,giveaways,shortHandedGoals,shortHandedAssists,blocked,plusMinus,evenTimeOnIce,shortHandedTimeOnIce,powerPlayTimeOnIce
0,2016020045,8468513,4,955,1,0,0,2.0,0,0,...,0,1.0,1.0,0,0,1.0,1,858,97,0
1,2016020045,8476906,4,1396,1,0,4,2.0,0,0,...,0,1.0,2.0,0,0,2.0,0,1177,0,219
2,2016020045,8474668,4,915,0,0,1,1.0,0,0,...,0,2.0,0.0,0,0,0.0,-1,805,0,110
3,2016020045,8473512,4,1367,3,0,0,0.0,0,2,...,27,0.0,0.0,0,0,0.0,-1,1083,19,265
4,2016020045,8471762,4,676,0,0,3,2.0,0,0,...,0,0.0,1.0,0,0,0.0,-1,613,63,0


In [6]:
game_goalie_stats = pd.read_csv(data_path+'game_goalie_stats.csv')
print(game_goalie_stats.shape)
game_goalie_stats.head()

(56656, 19)


,game_id,player_id,team_id,timeOnIce,assists,goals,pim,shots,saves,powerPlaySaves,shortHandedSaves,evenSaves,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,decision,savePercentage,powerPlaySavePercentage,evenStrengthSavePercentage
0,2016020045,8473607,4,1504,0,0,0,16,12,1,0,11,0,13,3,NaN,75.000000,33.333333,84.615385
1,2016020045,8473461,4,2011,0,0,0,11,9,1,0,8,0,10,1,L,81.818182,100.000000,80.000000
2,2016020045,8470645,16,3598,0,0,0,27,23,2,0,21,0,23,4,W,85.185185,50.000000,91.304348
3,2017020812,8468011,24,3696,0,0,0,33,30,1,2,27,3,28,2,W,90.909091,50.000000,96.428571
4,2017020812,8475215,7,3539,0,0,0,33,29,4,1,24,1,27,5,L,87.878788,80.000000,88.888889


In [21]:
game_player = pd.read_csv(data_path+'player_info.csv')
print(game_player.shape)
game_player.head()

(3925, 12)


,player_id,firstName,lastName,nationality,birthCity,primaryPosition,birthDate,birthStateProvince,height,height_cm,weight,shootsCatches
0,8466148,Marian,Hossa,SVK,Stará Lubovna,RW,1979-01-12 00:00:00,NaN,"6' 1""",185.42,207.0,L
1,8465058,Michal,Rozsival,CZE,Vlasim,D,1978-09-03 01:00:00,NaN,"6' 1""",185.42,210.0,R
2,8476906,Shayne,Gostisbehere,USA,Pembroke Pines,D,1993-04-20 01:00:00,FL,"5' 11""",180.34,180.0,L
3,8466285,Brian,Campbell,CAN,Strathroy,D,1979-05-23 01:00:00,ON,"5' 10""",177.80,192.0,L
4,8470607,Brent,Seabrook,CAN,Richmond,D,1985-04-20 01:00:00,BC,"6' 3""",190.50,220.0,R


## Merge other data 
- drop duplicates
- append columns to the 'game_stats'

### Merge games and game_stats

In [8]:
games = games.drop_duplicates()
games.columns
# selected columns: 'type', 'date_time_GMT', 'home_rink_side_start', 'venue', 'venue_time_zone_id', 
# 'venue_time_zone_offset', 'venue_time_zone_tz'

Index(['game_id', 'season', 'type', 'date_time_GMT', 'away_team_id',
       'home_team_id', 'away_goals', 'home_goals', 'outcome',
       'home_rink_side_start', 'venue', 'venue_link', 'venue_time_zone_id',
       'venue_time_zone_offset', 'venue_time_zone_tz'],
      dtype='object')

In [9]:
game_stats.drop_duplicates()
combined = game_stats.copy()
games_cols = games[['game_id', 'type', 'date_time_GMT', 'home_rink_side_start', 'venue', 'venue_time_zone_id', 'venue_time_zone_offset', 'venue_time_zone_tz']]
combined = combined.merge(games_cols, how='left', left_on='game_id', right_on='game_id')
print(combined.shape)
combined.head()

(52610, 24)


,game_id,team_id,HoA,won,settled_in,head_coach,goals,shots,hits,pim,...,takeaways,blocked,startRinkSide,type,date_time_GMT,home_rink_side_start,venue,venue_time_zone_id,venue_time_zone_offset,venue_time_zone_tz
0,2016020045,4,away,False,REG,Dave Hakstol,4.0,27.0,30.0,6.0,...,9.0,11.0,left,R,2016-10-19T00:30:00Z,right,United Center,America/Chicago,-5,CDT
1,2016020045,16,home,True,REG,Joel Quenneville,7.0,28.0,20.0,8.0,...,8.0,9.0,left,R,2016-10-19T00:30:00Z,right,United Center,America/Chicago,-5,CDT
2,2017020812,24,away,True,OT,Randy Carlyle,4.0,34.0,16.0,6.0,...,4.0,14.0,right,R,2018-02-07T00:00:00Z,left,KeyBank Center,America/New_York,-4,EDT
3,2017020812,7,home,False,OT,Phil Housley,3.0,33.0,17.0,8.0,...,6.0,14.0,right,R,2018-02-07T00:00:00Z,left,KeyBank Center,America/New_York,-4,EDT
4,2015020314,21,away,True,REG,Patrick Roy,4.0,29.0,17.0,9.0,...,5.0,20.0,left,R,2015-11-24T01:00:00Z,right,MTS Centre,America/Winnipeg,-5,CDT


In [10]:
combined.columns

Index(['game_id', 'team_id', 'HoA', 'won', 'settled_in', 'head_coach', 'goals',
       'shots', 'hits', 'pim', 'powerPlayOpportunities', 'powerPlayGoals',
       'faceOffWinPercentage', 'giveaways', 'takeaways', 'blocked',
       'startRinkSide', 'type', 'date_time_GMT', 'home_rink_side_start',
       'venue', 'venue_time_zone_id', 'venue_time_zone_offset',
       'venue_time_zone_tz'],
      dtype='object')

### Merge game_skater_stats and game_stats

In [11]:
game_skater_stats = game_skater_stats.drop_duplicates()
game_skater_stats.columns
# selected columns: timeOnIce, evenTimeOnIce, shortHandedTimeOnIce, powerPlayTimeOnIce

Index(['game_id', 'player_id', 'team_id', 'timeOnIce', 'assists', 'goals',
       'shots', 'hits', 'powerPlayGoals', 'powerPlayAssists', 'penaltyMinutes',
       'faceOffWins', 'faceoffTaken', 'takeaways', 'giveaways',
       'shortHandedGoals', 'shortHandedAssists', 'blocked', 'plusMinus',
       'evenTimeOnIce', 'shortHandedTimeOnIce', 'powerPlayTimeOnIce'],
      dtype='object')

In [12]:
skater_agg_cols = game_skater_stats.groupby(['game_id', 'team_id'], as_index=False)[['timeOnIce', 'evenTimeOnIce', 'shortHandedTimeOnIce', 'powerPlayTimeOnIce']].mean()
combined = combined.merge(skater_agg_cols, how='left', left_on=['game_id', 'team_id'], right_on=['game_id', 'team_id'])
print(combined.shape)
combined.head()

(52610, 28)


,game_id,team_id,HoA,won,settled_in,head_coach,goals,shots,hits,pim,...,date_time_GMT,home_rink_side_start,venue,venue_time_zone_id,venue_time_zone_offset,venue_time_zone_tz,timeOnIce,evenTimeOnIce,shortHandedTimeOnIce,powerPlayTimeOnIce
0,2016020045,4,away,False,REG,Dave Hakstol,4.0,27.0,30.0,6.0,...,2016-10-19T00:30:00Z,right,United Center,America/Chicago,-5,CDT,990.833333,841.388889,55.555556,93.888889
1,2016020045,16,home,True,REG,Joel Quenneville,7.0,28.0,20.0,8.0,...,2016-10-19T00:30:00Z,right,United Center,America/Chicago,-5,CDT,981.333333,836.777778,75.111111,69.444444
2,2017020812,24,away,True,OT,Randy Carlyle,4.0,34.0,16.0,6.0,...,2018-02-07T00:00:00Z,left,KeyBank Center,America/New_York,-4,EDT,1002.222222,879.611111,28.444444,94.166667
3,2017020812,7,home,False,OT,Phil Housley,3.0,33.0,17.0,8.0,...,2018-02-07T00:00:00Z,left,KeyBank Center,America/New_York,-4,EDT,999.222222,888.333333,75.333333,35.555556
4,2015020314,21,away,True,REG,Patrick Roy,4.0,29.0,17.0,9.0,...,2015-11-24T01:00:00Z,right,MTS Centre,America/Winnipeg,-5,CDT,986.666667,844.722222,53.333333,88.611111


In [13]:
combined.columns

Index(['game_id', 'team_id', 'HoA', 'won', 'settled_in', 'head_coach', 'goals',
       'shots', 'hits', 'pim', 'powerPlayOpportunities', 'powerPlayGoals',
       'faceOffWinPercentage', 'giveaways', 'takeaways', 'blocked',
       'startRinkSide', 'type', 'date_time_GMT', 'home_rink_side_start',
       'venue', 'venue_time_zone_id', 'venue_time_zone_offset',
       'venue_time_zone_tz', 'timeOnIce', 'evenTimeOnIce',
       'shortHandedTimeOnIce', 'powerPlayTimeOnIce'],
      dtype='object')

### Merge game_goalie_stats and game_stats

In [14]:
game_goalie_stats = game_goalie_stats.drop_duplicates()
game_goalie_stats.columns
# selected: team_id (is there a backup goalie?)

Index(['game_id', 'player_id', 'team_id', 'timeOnIce', 'assists', 'goals',
       'pim', 'shots', 'saves', 'powerPlaySaves', 'shortHandedSaves',
       'evenSaves', 'shortHandedShotsAgainst', 'evenShotsAgainst',
       'powerPlayShotsAgainst', 'decision', 'savePercentage',
       'powerPlaySavePercentage', 'evenStrengthSavePercentage'],
      dtype='object')

In [15]:
game_goalie_stats_subset = game_goalie_stats[['game_id', 'team_id']]
game_goalie_stats_subset = game_goalie_stats_subset.groupby(['game_id', 'team_id'])["team_id"].count().reset_index(name="goalie_count")
game_goalie_stats_subset['goalie_replacement'] = np.where(game_goalie_stats_subset['goalie_count'] == 2, "Yes", "No")
goalie_agg_cols = game_goalie_stats_subset[['game_id', 'team_id', 'goalie_replacement']]

combined = combined.merge(goalie_agg_cols, how='left', left_on=['game_id', 'team_id'], right_on=['game_id', 'team_id'])
print(combined.shape)
combined.head()

(52610, 29)


,game_id,team_id,HoA,won,settled_in,head_coach,goals,shots,hits,pim,...,home_rink_side_start,venue,venue_time_zone_id,venue_time_zone_offset,venue_time_zone_tz,timeOnIce,evenTimeOnIce,shortHandedTimeOnIce,powerPlayTimeOnIce,goalie_replacement
0,2016020045,4,away,False,REG,Dave Hakstol,4.0,27.0,30.0,6.0,...,right,United Center,America/Chicago,-5,CDT,990.833333,841.388889,55.555556,93.888889,Yes
1,2016020045,16,home,True,REG,Joel Quenneville,7.0,28.0,20.0,8.0,...,right,United Center,America/Chicago,-5,CDT,981.333333,836.777778,75.111111,69.444444,No
2,2017020812,24,away,True,OT,Randy Carlyle,4.0,34.0,16.0,6.0,...,left,KeyBank Center,America/New_York,-4,EDT,1002.222222,879.611111,28.444444,94.166667,No
3,2017020812,7,home,False,OT,Phil Housley,3.0,33.0,17.0,8.0,...,left,KeyBank Center,America/New_York,-4,EDT,999.222222,888.333333,75.333333,35.555556,No
4,2015020314,21,away,True,REG,Patrick Roy,4.0,29.0,17.0,9.0,...,right,MTS Centre,America/Winnipeg,-5,CDT,986.666667,844.722222,53.333333,88.611111,No


In [16]:
combined.columns

Index(['game_id', 'team_id', 'HoA', 'won', 'settled_in', 'head_coach', 'goals',
       'shots', 'hits', 'pim', 'powerPlayOpportunities', 'powerPlayGoals',
       'faceOffWinPercentage', 'giveaways', 'takeaways', 'blocked',
       'startRinkSide', 'type', 'date_time_GMT', 'home_rink_side_start',
       'venue', 'venue_time_zone_id', 'venue_time_zone_offset',
       'venue_time_zone_tz', 'timeOnIce', 'evenTimeOnIce',
       'shortHandedTimeOnIce', 'powerPlayTimeOnIce', 'goalie_replacement'],
      dtype='object')